In [ ]:
deploy_phase = "local"
execution_date = '2024-07-06T00:00:00+09:00'

# Notebook Initialization

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import sys
# sys.path.insert(0, "/workspace/ojitong")

# Global Const

In [ ]:
from datetime import datetime

def convert_to_datetime(date_str: str) -> datetime:
    return datetime.fromisoformat(date_str)

today_datetime = convert_to_datetime(execution_date)
today_datetime

# Knowledge Selection

In [ ]:
# import onnxruntime as ort
# import numpy as np
# from kobert import get_onnx_kobert_model
# from transformers import BertTokenizer

# # ONNX 모델 및 세션 초기화
# onnx_path = get_onnx_kobert_model()
# sess = ort.InferenceSession(onnx_path)

# import spacy
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity

# # Load the Korean model from spaCy
# nlp = spacy.load('ko_core_news_lg')

# # Load KoBERT model and tokenizer
# tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# # Define related and action words
# related_words = [
#     "지하철", "교통", "운행", "노선", "승객", "출근", "전철", "메트로", "지하철역", "승강장", "지하철차량",
#     "대중교통", "교통체계", "교통망", "환승", "전용차선","사고",
# ]

# action_words = [
#     "파업", "지연", "연착", "중단", "정지", "혼잡", "운행지연", "운행중단", "서비스중단", "파업예고",
#     "노동쟁의", "노조활동", "노사협상", "안전점검", "사고", "충돌", "부상", "사망", "운행변경",
#     "노선변경", "시간표변경", "대체교통", "운행재개", "시위","대피"
# ]

# # Function to check dependency and entities
# def check_dependency_and_entities(text):
#     doc = nlp(text)
#     for token in doc:
#         if token.lemma_ in action_words:
#             neighborhood = list(token.children) + list(token.ancestors) + list(token.subtree)
#             for neighbor in neighborhood:
#                 if neighbor.lemma_ in related_words:
#                     if any(child.dep_ == "neg" for child in neighbor.children):
#                         continue
#                     return True
#                 if neighbor.pos_ in ["ADP", "SCONJ"]:
#                     connected_words = [child.lemma_ for child in neighbor.children]
#                     if set(connected_words).intersection(related_words):
#                         return True
#     return False

# # Function to get BERT embeddings
# def get_bert_embedding(text):
#     # 텍스트를 토크나이즈
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
#     input_ids = inputs['input_ids'].detach().numpy()
#     attention_mask = inputs['attention_mask'].detach().numpy()
#     token_type_ids = inputs['token_type_ids'].detach().numpy()

#     # ONNX 모델을 사용한 추론
#     len_seq = input_ids.shape[1]
#     outputs = sess.run(
#         None,
#         {
#             'input_ids': input_ids,
#             'token_type_ids': token_type_ids,
#             'input_mask': attention_mask,
#             'position_ids': np.array(range(len_seq))
#         }
#     )

#     # 마지막 인코딩 레이어의 출력값 평균을 계산
#     last_hidden_state = outputs[-2]  # ONNX 출력 중 마지막 인코딩 레이어에 해당
#     mean_embeddings = np.mean(last_hidden_state, axis=1)
#     return mean_embeddings

# # Function to check topic relevance using BERT embeddings
# def check_topics(text, reference_texts):
#     text_embedding = get_bert_embedding(text)
#     reference_embeddings = [get_bert_embedding(ref) for ref in reference_texts]
#     similarities = [cosine_similarity(text_embedding, ref_emb)[0][0] for ref_emb in reference_embeddings]
#     return max(similarities) > 0.8  # Adjusted threshold for relevance

# # Load your dataframe (assuming df is already defined and has a 'text' column)
# reference_texts = [
#     "지하철 파업으로 인한 운행 중단",
#     "지하철 사고 발생",
#     "지하철 연착 문제",
#     "지하철 노선 변경",
#     "지하철 운행 재개",
#     "지하철 서비스 중단",
#     "지하철 안전 점검",
#     "지하철 충돌 사고",
#     "지하철 승객 혼잡",
#     "지하철 출근 시간 문제",
#     "지하철 노조 파업",
#     "지하철 운행 변경",
#     "지하철 대체 교통"
# ]

# # Check if the document is relevant
# def is_relevant_document(title):
#     return check_dependency_and_entities(title) or check_topics(title, reference_texts)

# # Assuming df is already defined and has a 'title' column
# df['is_relevant'] = df['title'].apply(is_relevant_document)
# df = df[df['is_relevant']]

In [ ]:
# df

In [ ]:
import pandas as pd
from data.utils.file import generate_s3_path
s3_path = generate_s3_path(today_datetime=today_datetime)
df = pd.read_csv(s3_path)

In [ ]:
df

In [ ]:
# 1. title 추출
df['indexed_title'] = df.index.astype(str) + ": " + df['title']
titles = df['indexed_title'].tolist()

In [ ]:
# 2. Selection News 태스크 실행
from data.tasks.news_selection_task import SelectionNewsTask
task = SelectionNewsTask()
indices = task.execute(titles)

In [ ]:
valid_indices = [i for i in indices if i < len(df)]
selected_df = df.iloc[valid_indices]

In [ ]:
selected_df

In [ ]:
# 3. 파일 S3 업로드
s3_path = generate_s3_path(prefix="news_data_selected", today_datetime=today_datetime)
selected_df.to_csv(s3_path, index=False)
print(f"*** Data saved to {s3_path} ***")

In [ ]:
s3_path